# Ottomatic Reply Project

In [1]:
!pip install langsmith mysql.connector

In [2]:
# Import libraries
from google import genai
from google.cloud import storage
from google.cloud import language_v2
from google.genai import types
from google.genai.types import HttpOptions
from langsmith import traceable
from langsmith.run_helpers import get_current_run_tree
from typing import List
import re, spacy
import datetime
import os
import json
import uuid

In [3]:
# Sample customer-service e-mails with tags
#Training Data

#[Product Consultation | Tone: Neutral | Urgency: Low]
email1 = """Subject: Question about the Nebula-X Smart Projector before I buy

Hi team,

I’m Maria-Luisa Ortega and I’m really excited about the Nebula-X projector! A quick check before I place my order:

* Will it handle 220 V power in Spain?
* Is it compatible with my Samsung Galaxy S24 via USB-C?

If you need to reach me: mlortega1987@gmail.com, mobile +34 612 445 778.  
Shipping would go to Calle Mayor 12, 2-B, 28013 Madrid, Spain.

Thanks a lot—looking forward to your advice!

Best,  
María-Luisa
"""

#[Order Support | Tone: Unhappy | Urgency: Medium]
email2 = """Subject: Trouble completing checkout – “Payment declined” error

Hello,

My name is Jonathan K. Barrett (phone +1 646-555-0194) and I’m trying to order Order ID draft #TEMP-789312 for the Tempo Fitness Watch.  
Your site keeps rejecting my Visa (•••• •••• •••• 4389 exp 11/27) and PayPal isn’t loading. Could someone walk me through a fix?  
I really want this before my hiking trip.

Billing address: 338 Hudson St, Apt 4C, New York, NY 10014, USA.

Thanks!

Jon
"""

#[Technical Assistance | Tone: Neutral | Urgency: Medium]
email3 = """Subject: Installation help for Aero-Max security camera

Hi Support,

I’m Lin Wei (林伟), order #AM-554-993. I followed the quick-start guide, but the camera won’t connect to my 5 GHz Wi-Fi. Router is a TP-Link AX50.

Could you call me at +86 139-2211-0444 or reply to linwei.cam@outlook.cn with advanced setup steps?

Regards,  
Wei  
Address: 23-2 Guangfu Rd, Huangpu, Guangzhou 510623, China
"""

#[Shipping & Delivery Updates | Tone: Unhappy | Urgency: Medium]
email4 = """Subject: Where is package #TRK-998776543DE? Leaving town soon!

Dear team,

I’m leaving on June 15 and my parcel still shows “Label created.”  
Recipient: Dr. Margot Klein, Gartenweg 8, 55116 Mainz, Germany, phone +49 151 6888 9900.  
Please confirm the revised ETA or upgrade to express—happy to pay the difference.

Feeling a bit stressed; would appreciate quick clarification.

Kind regards,  
Margot
"""

email5 = """Subject: Return request – wrong color delivered (Order #EU-445821)

Hello Returns Team,

I’m Pierre Lefèvre, order arrived today at 27 Rue des Lilas, 33000 Bordeaux, France but you sent the *Crimson Red* headphones instead of *Classic Black*.

Please advise on getting a prepaid label or arranging a swap.  
Email: p.lefevre@free.fr, phone +33 6 78 55 22 11.

Merci beaucoup,  
Pierre
"""

email6 = """Subject: DEFECTIVE air fryer exploded oil – urgent replacement demanded

To whom it may concern,

This is Sophia Hernandez (Order #MX-24681012). Last night the “Quick-Crisp 6 L” air fryer sprayed hot oil across my kitchen, nearly burning my son!  
Serial No: QC6-A1B2-C3D4.

I need an *immediate* replacement or full refund plus compensation for cleaning costs (receipt attached). Reach me at sophia.hdz@email.com or +52 55 2312 7766.  
Address: Calle Lago Garda 15, 1-A, 11520 CDMX, Mexico.

I’m furious—please act fast.

— Sophia
"""

email7 = """Subject: Double charge on credit card - invoice INV-2025-5566

Hi Billing,

I’m Khalid Al-Mazrouei, customer #C102233. My Mastercard (•••• •••• •••• 8120) was charged AED 1,299 twice on 5 June 2025.  
Please reverse the extra transaction.

Statement attached. Contact: +971 50 123 4567, khalid.am@protonmail.com.  
Postal: Villa 42, Al Raha Gardens, Abu Dhabi, UAE.

Regards,  
Khalid
"""

email8 = """Subject: Cannot reset password – account locked after 3 attempts

Hello,

I’m Emma-Jane O’Neill, username ejoneill88. The reset e-mail never arrives (checked spam).  
Could you manually reset or update the address to emma-jane.oneill@icloud.com?

Phone: +353 87 665 9020  
Current address on file: 74 Sandymount Rd, Dublin 4, Ireland.

Thanks,  
Emma-Jane
"""

email9 = """Subject: Missing loyalty points for last week’s purchase 🤔

Hey Rewards Crew!

Love the store ☺️ but my DragonFly Points balance didn’t update after Order #UK-PO-98456 (£289.99 on 4 June).  
Member ID DF-448822. Could you add the points and confirm?  

Reach me: luke.bennett@gmail.com, mobile +44 7900 123 456.  
Shipping address: Flat 7, 10 Jubilee Pl, London SW3 3TD, UK.

Cheers,  
Luke
"""

email10 = """Subject: Feedback on recent chatbot experience – could be better

Hi Customer Care,

I’m Dr. Aisha Yusuf, e-mail a.yusuf@postgrad.edu.ng, phone +234 803 555 0190.  
After yesterday’s chat session (Case ID CC-4321), I felt the bot looped the same FAQs and wasted 20 minutes.

Overall I like your products, but human escalation should happen sooner.  
Happy to discuss; my address: 12 Admiralty Way, Lekki Phase 1, Lagos, Nigeria.

Thank you for listening!

Best regards,  
Aisha
"""

# All in one list if you prefer iterable access:
sample_emails = [
    email1, email2, email3, email4, email5,
    email6, email7, email8, email9, email10,
]

In [4]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "Project-Southafrica"

## PII (Redaction)

In [5]:
REGEX_PATTERNS: List[tuple[str, str]] = [
    (r"\b[\w.-]+@[\w.-]+\.\w+\b", "[REDACTED]"),
    (r"\b(?:\+?\d{1,3})?[-.\s]?\(?\d{2,4}\)?[-.\s]?\d{3,4}[-.\s]?\d{3,4}\b", "[REDACTED]"),
    (r"\b\d{4}[-\s]?\d{4}[-\s]?\d{4}[-\s]?\d{4}\b", "[REDACTED]"),
    (r"\bORD-\d{6,8}\b", "[REDACTED]"),
]

nlp = spacy.load("en_core_web_md", disable=["tagger", "parser", "lemmatizer"])

@traceable
def redact(text: str) -> str:
    """
    Redacts sensitive information (Personally Identifiable Information - PII) from a given text.

    This function performs redaction in two main steps:
    1.  **Regex-based Redaction:** Applies a predefined set of regular expressions
        to identify and replace common patterns like email addresses, phone numbers,
        credit card numbers, and specific order IDs with generic '[REDACTED]' placeholders.
    2.  **Named Entity Recognition (NER) Redaction:** Uses the spaCy library to
        identify and redact specific types of named entities from the text,
        including:
        -   **LOC** (Geographical locations)
        -   **PERSON** (People's names)
        -   **GPE** (Geopolitical entities like countries, cities, states)
        Entities are processed in reverse ord
        er to avoid issues with changing
        string indices during redaction.

    Args:
        text (str): The input text (e.g., customer email) from which PII needs to be redacted.

    Returns:
        str: The redacted text with identified PII replaced by '[REDACTED]' placeholders.
    """
    for pattern, repl in REGEX_PATTERNS:
        text = re.sub(pattern, repl, text, flags=re.I)
    doc = nlp(text)
    for ent in reversed(doc.ents):
        if ent.label_ in {"LOC", "PERSON", "GPE", "NORP", "FAC"}:
            text = text[:ent.start_char] + f"[REDACTED]" + text[ent.end_char:]
    return text

In [6]:
for email in sample_emails:
    print(redact(email))

Subject: Question about the Nebula-X Smart Projector before I buy

Hi team,

I’m [REDACTED] and I’m really excited about the Nebula-X projector! A quick check before I place my order:

* Will it handle 220 V power in [REDACTED]?
* Is it compatible with my Samsung Galaxy S24 via USB-C?

If you need to reach me: [REDACTED], mobile +[REDACTED].  
Shipping would go to [REDACTED] Mayor 12, 2-B, 28013 [REDACTED], [REDACTED].

Thanks a lot—looking forward to your advice!

Best,  
María-Luisa

Subject: Trouble completing checkout – “Payment declined” error

Hello,

My name is [REDACTED] (phone +[REDACTED]) and I’m trying to order Order ID draft #TEMP-789312 for the Tempo Fitness Watch.  
Your site keeps rejecting my Visa (•••• •••• •••• 4389 exp 11/27) and PayPal isn’t loading. Could someone walk me through a fix?  
I really want this before my hiking trip.

Billing address: 338 [REDACTED], Apt 4C, [REDACTED], NY 10014, [REDACTED].

Thanks!

[REDACTED]

Subject: Installation help for Aero-Max 


Subject: Question about the Nebula-X Smart Projector before I buy

Hi team,

I’m [REDACTED_PERSON] and I’m really excited about the Nebula-X projector! A quick check before I place my order:

* Will it handle 220 V power in [REDACTED_GPE]?
* Is it compatible with my Samsung Galaxy S24 via USB-C?

If you need to reach me: [REDACTED_EMAIL], mobile +[REDACTED_PHONE].  
Shipping would go to [REDACTED_PERSON] Mayor 12, 2-B, 28013 [REDACTED_GPE], [REDACTED_GPE].

Thanks a lot—looking forward to your advice!

Best,  
María-Luisa



Subject: Trouble completing checkout – “Payment declined” error

Hello,

My name is [REDACTED_PERSON] (phone +[REDACTED_PHONE]) and I’m trying to order Order ID draft #TEMP-789312 for the Tempo Fitness Watch.  
Your site keeps rejecting my Visa (•••• •••• •••• [REDACTED_DATE] exp 11/27) and PayPal isn’t loading. Could someone walk me through a fix?  
I really want this before my hiking trip.

Billing address: 338 [REDACTED_PERSON], Apt 4C, [REDACTED_GPE], NY 10014

# Classification

In [7]:
classification_prompt = """
You are an email classification system for a customer support team. 
Categorize the following email into ONE of these 10 support groups:

1. **Product Consultation (Presales)**: Questions about product features, compatibility, or usage before purchase.
2. **Order Support (Presales)**: Issues with ordering, payment options, or shipping selection.
3. **Technical Assistance (Presales/Aftersales)**: Help with installation, setup, or troubleshooting.
4. **Shipping and Delivery Updates (Aftersales)**: Status, tracking, or delivery date inquiries.
5. **Returns and Exchanges Management (Aftersales)**: Returns, exchanges, or refund requests.
6. **Claims and Product Defects (Aftersales)**: Reports of defective/damaged items.
7. **Payment and Billing Support (Aftersales)**: Invoices, payment issues, or credit problems.
8. **Customer Account Support (Aftersales)**: Password resets, account changes.
9. **Loyalty Programs and Discounts (Presales/Aftersales)**: Vouchers, discounts, or loyalty programs.
10. **Customer Feedback and Complaints (Aftersales)**: Complaints or feedback submissions.

**Rules:**
- Respond ONLY with the exact category name (e.g., "Shipping and Delivery Updates (Aftersales)").
- If unsure, pick the most likely option.

**Email to classify:**
"{email_text}"

**Classification:**
"""

@traceable
async def classify_email(email_text: str, classification_prompt: str) -> str:
    """
    Classifies an email's content using the Google Gemini model
    based on a provided classification prompt.

    This asynchronous function initializes a Google AI client and sends the
    email text along with a system instruction (the classification prompt)
    to the Gemini model for content generation, specifically for classification.
    The response text from the model is then returned, with any trailing
    newline characters removed.

    Args:
        email_text (str): The content of the email to be classified. This text
                          will be sent to the Gemini model for analysis.
        classification_prompt (str): A string containing the system instruction
                                     that guides the Gemini model on how to classify 
                                     the email content.

    Returns:
        str: The classified category or response text generated by the Gemini model.
             Trailing newline characters are removed from the response.
    """

    client = genai.Client(
        http_options=HttpOptions(api_version="v1"), 
        vertexai=True,
        project="ogcs-av8t-ailaboratory",
        location="europe-west1"
    )

    response = await client.aio.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=email_text,
        config=types.GenerateContentConfig(
            system_instruction=[classification_prompt]
        ),
    )

    return response.text.rstrip("\n")

# Sentiment Analysis

In [8]:
@traceable
def analyze_sentiment(email_text: str) -> dict:
    """
    Analyzes the sentiment of a given text using the Google Cloud Natural Language API
    and returns a detailed dictionary containing sentiment scores, magnitude,
    language, and sentiment breakdown by sentence.

    This function utilizes the `language_v2.LanguageServiceClient` to process the
    input text and determine its overall sentiment, as well as the sentiment of
    individual sentences within the text.

    The sentiment is categorized into "Very unhappy", "Unhappy", "Neutral",
    "Happy", and "Very Happy" based on predefined score ranges.

    Args:
        text (str): The input text (e.g., content of an email or message) to be
                    analyzed for sentiment.

    Returns:
        dict: A dictionary containing the sentiment analysis results.
              The structure of the dictionary is as follows:
              {
                  "sentiment_category": str,  # Overall sentiment ("Very unhappy", "Happy", etc.)
                  "score": float,             # Overall sentiment score (-1.0 to 1.0)
                  "magnitude": float,         # Overall sentiment magnitude (0.0 to +inf)
                  "language": str,            # Detected language of the text (e.g., "en")
                  "sentences": list           # List of dictionaries, each representing a sentence:
                      [
                          {
                              "text": str,          # The content of the sentence
                              "score": float,       # Sentiment score for the sentence
                              "magnitude": float,   # Sentiment magnitude for the sentence
                              "sentiment_label": str # Categorized sentiment for the sentence
                          },
                          ...
                      ]
              }
    """

    client = language_v2.LanguageServiceClient()

    document = {
        "content": email_text,
        "type_": language_v2.Document.Type.PLAIN_TEXT,
    }

    encoding_type = language_v2.EncodingType.UTF8

    response = client.analyze_sentiment(
        request={"document": document, "encoding_type": encoding_type}
    )

    score = response.document_sentiment.score
    magnitude = response.document_sentiment.magnitude

    def interpret_score(score: float) -> str:
        if score <= -0.35:
            return "Very unhappy"
        elif score <= -0.1:
            return "Unhappy"
        elif score < 0.25:
            return "Neutral"
        elif score < 0.75:
            return "Happy"
        else:
            return "Very Happy"

    result = {
        "sentiment_category": interpret_score(score),
        "score": score,
        "magnitude": magnitude,
        "language": response.language_code,
        "sentences": []
    }

    for sentence in response.sentences:
        sentence_score = sentence.sentiment.score
        result["sentences"].append({
            "text": sentence.text.content,
            "score": sentence_score,
            "magnitude": sentence.sentiment.magnitude,
            "sentiment_label": interpret_score(sentence_score)
        })

    return result


# Urgency

In [9]:
@traceable
def define_urgency(category: str, sentiment: str) -> str:
    """
    Calculates an urgency score based on the category of the customer interaction
    and the sentiment expressed by the customer.

    The urgency score is determined by:
    1.  **Category:** Different categories are assigned varying urgency points.
        -   Aftersales categories (Shipping, Returns, Claims, Payment) add 3 points.
        -   Presales/Aftersales categories (Product Consultation, Order Support,
            Technical Assistance, Customer Account) add 2 points.
        -   Less urgent categories (Loyalty Programs, Customer Feedback) add 1 point.
    2.  **Sentiment:** Customer sentiment further modifies the urgency.
        -   "Very unhappy" sentiment adds 2 points.
        -   "Unhappy" sentiment adds 1 point.

    The final urgency score is the sum of points from both category and sentiment.

    Args:
        category (str): The classification of the customer interaction (e.g.,
                        "Shipping and Delivery Updates (Aftersales)",
                        "Product Consultation (Presales)").
        sentiment (str): The sentiment expressed by the customer (e.g.,
                         "Very unhappy", "Unhappy", "Neutral", "Happy").

    Returns:
        int: An integer representing the calculated urgency score.
             A higher score indicates greater urgency.
    """

    urgency_score = 0

    if category in ["Shipping and Delivery Updates (Aftersales)", 
                    "Returns and Exchanges Management (Aftersales)",
                    "Claims and Product Defects (Aftersales)",
                   "Payment and Billing Support (Aftersales)"]:
        urgency_score += 3
    elif category in ["Product Consultation (Presales)", 
                      "Order Support (Presales)",
                     "Technical Assistance (Presales/Aftersales)",
                      "Customer Account Support (Aftersales)"]:
        urgency_score += 2
    elif category in ["Loyalty Programs and Discounts (Presales/Aftersales)", 
                      "Customer Feedback and Complaints (Aftersales)"]:
        urgency_score += 1

    if sentiment == "Very unhappy":
        urgency_score += 2
    elif sentiment == "Unhappy":
        urgency_score += 1

    return urgency_score

# Draft Reply

In [10]:
@traceable
async def create_draft_reply (email_text: str) -> str:
    """
    Generates a helpful, brand-aligned draft reply template for a customer email
    using the Google Gemini 2.0 Flash model.

    This function constructs a detailed prompt that instructs the AI to act as
    a customer support agent from the Otto Group, emphasizing brand alignment
    and strict avoidance of Personal Identifiable Information (PII) in the
    generated reply. It then sends this prompt, along with the customer's
    original email text, to the `gemini-2.0-flash-001` model via the
    Google Generative AI client.

    The model's response, which is the drafted reply, is returned as a string.

    Args:
        email_text (str): The full content of the customer's original email,
                          which the AI will use as context to generate the reply.

    Returns:
        str: A brand-aligned, PII-free draft reply template generated by the
             Gemini model.
    """
    draft_prompt = f"""You are a customer support agent.
    You are generating a brand-aligned draft reply template from the Otto Group in response to a customer message.
    Do not include any personal information about the customer in the template.

    Here is the customer's original email:
    ---
    {email_text}
    ---

    Based on all this information, please write a draft reply.
    """
    client = genai.Client(
        http_options=HttpOptions(api_version="v1"), 
        vertexai=True,
        project="ogcs-av8t-ailaboratory",
        location="europe-west1"
    )
    response = await client.aio.models.generate_content(
        model='gemini-2.0-flash-001',
        contents= draft_prompt,

    )
    return response.text

# Put all together

In [11]:
@traceable
async def ottomation(original_email_text: str, 
                    email_address: str) -> {}:
    """
    Orchestrates a comprehensive email processing workflow, from redaction and
    classification to sentiment analysis, urgency assessment, and draft reply generation.

    This asynchronous function integrates several specialized sub-functions to
    automate the initial handling of customer emails. It takes an original email
    and the sender's email address, processes the email content through various
    AI and rule-based steps, and compiles all generated insights into a structured
    dictionary.

    The workflow includes:
    1.  **Redaction:** Hides sensitive information from the original email.
    2.  **Classification:** Determines the primary support team/category for the email.
    3.  **Sentiment Analysis:** Evaluates the overall sentiment (e.g., happy, unhappy)
        and magnitude of the email content.
    4.  **Urgency Definition:** Calculates an urgency score based on the email's category
        and sentiment.
    5.  **Draft Reply Creation:** Generates a brand-aligned, PII-free draft response.
    6.  **Timestamping:** Records when the processing occurred.

    Args:
        original_email_text (str): The complete, raw content of the customer's email.
        email_address (str): The email address of the sender.

    Returns:
        dict: A dictionary containing all the processed information and insights
              derived from the email. The dictionary includes:
              - `timestamp` (str): The date and time of processing (dd/mm/YYYY HH:MM).
              - `email_address` (str): The sender's email address.
              - `subject` (str): Subject retrieved from the first line in the redacted email.
              - `original_email_text` (str): The original, unredacted email content.
              - `redacted_email_text` (str): The email content after PII redaction.
              - `support_team` (str): The classified support team or category.
              - `sentiment_category` (str): The categorized sentiment (e.g., "Happy", "Unhappy").
              - `sentiment_score` (float): The numerical sentiment score (-1.0 to 1.0).
              - `sentiment_magnitude` (float): The numerical sentiment magnitude.
              - `urgency` (int): The calculated urgency score for the email.
              - `draft_reply` (str): The AI-generated draft response.
              - `answered` (bool): A flag indicating if the email has been answered (initially False).
    """
    current_run = get_current_run_tree()

    # Access the trace_id and run_id
    trace_id = current_run.trace_id
   
    redacted_email_text = redact(original_email_text)
    support_team = await classify_email(redacted_email_text, classification_prompt) 
    sentiment = analyze_sentiment(redacted_email_text)
    urgency = define_urgency(support_team, str(sentiment["sentiment_category"]))
    draft_reply = await create_draft_reply(redacted_email_text)

    ### Getting the Timestamp of now
    now = datetime.datetime.now()
    formatted_date = now.strftime("%d/%m/%Y")
    formatted_time = now.strftime("%H:%M")

    ### Getting the subject from the subject line
    match = re.search(r"Subject:\s*(.*?)(?=\n|$)", redacted_email_text, re.IGNORECASE)
    if match:
        # If a subject is found, strip any leading/trailing whitespace
        subject = match.group(1).strip()

    result = {
        "timestamp": f"{formatted_date} {formatted_time}",
        "email_address": email_address,
        "subject": subject,
        "original_email_text": original_email_text,
        "redacted_email_text": redacted_email_text,
        "support_team": support_team,
        "sentiment_category": sentiment["sentiment_category"],
        "sentiment_score": sentiment["score"],
        "sentiment_magnitude": sentiment["magnitude"],
        "urgency": urgency,
        "draft_reply": draft_reply,
        "answered": False
    }

    return result

   # Upload Data

In [12]:
def convert_to_json(data: dict) -> str:
    """
    Converts a dictionary to a JSON string.

    Args:
        data (dict): The dictionary to convert.

    Returns:
        str: The JSON string representation of the dictionary.
    """
    return json.dumps(data, indent=2, ensure_ascii=False)

def upload_data_to_gcs(bucket_name: str, file_name: str, data: str):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_string(data)
    print(f"Data uploaded to {bucket_name}/{file_name}")


all_results = []

for email_text in sample_emails:
    unique_email = f"{uuid.uuid4().hex}@example.com"  # z.B. 'e2f6c59ef4d14867a3822fa08d5b9c7c@example.com'
    
    result = await ottomation(
        original_email_text=email_text,
        email_address=unique_email
    )
    all_results.append(result)

json_data = convert_to_json(all_results)
upload_data_to_gcs("hackathon-team2-bucket", "all_customer_support_analysis.json", json_data)


Subject: Question about the Nebula-X Smart Projector before I buy

Hi team,

I’m [REDACTED] and I’m really excited about the Nebula-X projector! A quick check before I place my order:

* Will it handle 220 V power in [REDACTED]?
* Is it compatible with my Samsung Galaxy S24 via USB-C?

If you need to reach me: [REDACTED], mobile +[REDACTED].  
Shipping would go to [REDACTED] Mayor 12, 2-B, 28013 [REDACTED], [REDACTED].

Thanks a lot—looking forward to your advice!

Best,  
María-Luisa

Subject: Trouble completing checkout – “Payment declined” error

Hello,

My name is [REDACTED] (phone +[REDACTED]) and I’m trying to order Order ID draft #TEMP-789312 for the Tempo Fitness Watch.  
Your site keeps rejecting my Visa (•••• •••• •••• 4389 exp 11/27) and PayPal isn’t loading. Could someone walk me through a fix?  
I really want this before my hiking trip.

Billing address: 338 [REDACTED], Apt 4C, [REDACTED], NY 10014, [REDACTED].

Thanks!

[REDACTED]

Subject: Installation help for Aero-Max 

! pip install google-genai